In [1]:
import numpy as np
import pandas as pd
import re

from sklearn.preprocessing import LabelEncoder

from keras.applications.vgg16 import VGG16, preprocess_input

from tensorflow.keras.preprocessing import image

In [2]:
!nvidia-smi

Wed Mar  6 20:05:14 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
#Load data frpm kaggle
from google.colab import files
files.upload()

%pip install -q kaggle
!rm -r ~/.kaggle
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
#!kaggle datasets list
!kaggle datasets download dqhdqmcttdqx/visuelle2 --unzip

Saving kaggle.json to kaggle.json
rm: cannot remove '/root/.kaggle': No such file or directory
100% 2.09G/2.09G [00:26<00:00, 129MB/s]
100% 2.09G/2.09G [00:26<00:00, 86.3MB/s]


In [6]:
data = pd.read_csv("/content/visuelle2/sales.csv")
print(data.head())
print(data.shape)

   Unnamed: 0  external_code  retail season     category   color  \
0           0              5      36   SS17  long sleeve    grey   
1           1              2      51   SS17  long sleeve  violet   
2           2              5      10   SS17  long sleeve    grey   
3           3              9      41   SS17     culottes  yellow   
4           4              5      13   SS17  long sleeve    grey   

       image_path       fabric release_date  restock  ...    2    3    4    5  \
0  PE17/00005.png      acrylic   2016-11-28       22  ...  1.0  1.0  2.0  1.0   
1  PE17/00002.png      acrylic   2016-11-28       17  ...  1.0  0.0  0.0  2.0   
2  PE17/00005.png      acrylic   2016-11-28       15  ...  1.0  0.0  1.0  1.0   
3  PE17/00009.png  scuba crepe   2016-11-28       32  ...  1.0  1.0  0.0  0.0   
4  PE17/00005.png      acrylic   2016-11-28       26  ...  4.0  0.0  3.0  0.0   

     6    7    8    9   10   11  
0  0.0  0.0  2.0  0.0  0.0  0.0  
1  0.0  0.0  0.0  1.0  1.0  0.0  
2 

In [9]:
# Encode categorical variables using LabelEncoder
le = LabelEncoder()
data["category_encoded"] = le.fit_transform(data["category"])
data["color_encoded"] = le.fit_transform(data["color"])
data["fabric_encoded"] = le.fit_transform(data["fabric"])

# Encode images using a pre-trained image model (optimized for multiple occurrences)
image_features_dict = {}  # Store features for efficient retrieval

# Extract features using VGG16
model = VGG16(weights="imagenet", include_top=False)

def encode_image(image_path):
    if image_path not in image_features_dict:
        # Load and preprocess the image (only if not already encoded)
        path = '/content/visuelle2/images/' + image_path
        if image_path == 'AI19/04442.png':
          path = '/content/04442.png'
        img = image.load_img(path, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)

        # Extract features using the pre-defined model
        features = model.predict(x, verbose=0).flatten()  # Flatten directly

        # Store features for future retrieval
        image_features_dict[image_path] = features
        if int(len(image_features_dict))%500 == 0:
          print ("step: ", len(image_features_dict))

    return image_features_dict[image_path]  # Retrieve stored features

# Apply encoding efficiently using the dictionary
data["image_features"] = data["image_path"].apply(encode_image)

step:  500


  0%|          | 0/106850 [05:23<?, ?it/s]


step:  1000
step:  1500
step:  2000
step:  2500
step:  3000
step:  3500
step:  4000
step:  4500
step:  5000


In [10]:
data.head()

,Unnamed: 0,external_code,retail,season,category,color,image_path,fabric,release_date,restock,...,6,7,8,9,10,11,category_encoded,color_encoded,fabric_encoded,image_features
0,0,5,36,SS17,long sleeve,grey,PE17/00005.png,acrylic,2016-11-28,22,...,0.0,0.0,2.0,0.0,0.0,0.0,11,4,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,1,2,51,SS17,long sleeve,violet,PE17/00002.png,acrylic,2016-11-28,17,...,0.0,0.0,0.0,1.0,1.0,0.0,11,7,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,2,5,10,SS17,long sleeve,grey,PE17/00005.png,acrylic,2016-11-28,15,...,1.0,1.0,1.0,0.0,0.0,1.0,11,4,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,3,9,41,SS17,culottes,yellow,PE17/00009.png,scuba crepe,2016-11-28,32,...,0.0,1.0,0.0,1.0,0.0,0.0,1,9,49,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,4,5,13,SS17,long sleeve,grey,PE17/00005.png,acrylic,2016-11-28,26,...,2.0,1.0,0.0,0.0,0.0,0.0,11,4,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [48]:
img_encoded = data[['image_path','image_features']].drop_duplicates(subset='image_path').reset_index(drop=True)
print(img_encoded.head())


,image_path,image_features
0,PE17/00005.png,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,PE17/00002.png,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,PE17/00009.png,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,PE17/00008.png,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,PE17/00016.png,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...
5350,AI19/05573.png,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5351,AI19/05576.png,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5352,AI19/04878.png,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5353,AI19/04884.png,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [11]:
data.to_csv('data.csv')

In [12]:
def cleanString(input_str):

    # Remove ellipsis and extra spaces
    cleaned_str = re.sub(r'\.{3}', '', input_str)
    cleaned_str = ' '.join(cleaned_str.split())

    # Add a zero after the decimal point for single-digit integers
    cleaned_str = cleaned_str.replace('. ', '.0 ')

    # Replace the last '...' with ']'
    cleaned_str = cleaned_str.replace('...', ']')

    input_str = ' '.join(cleaned_str.split())

    # Remove brackets and split by space
    split_values = input_str[1:-1].split()

    # Convert to numpy array
    result_array = np.array(split_values, dtype=float)

    return result_array


In [13]:
# Load the CSV file
df = pd.read_csv("/content/data.csv")

# Extract the image features array
image_features_list = df["image_features"].tolist()

listOfArrays = []
for str in image_features_list:

    # Replace multiple spaces with a single space
    input_str = ' '.join(str.split())
    result_array = cleanString(input_str)
    listOfArrays.append(result_array)

df['image_features'] = listOfArrays

df =df[["external_code", "retail", "category_encoded",
                    "fabric_encoded", "color_encoded", '0', '1', '2', '3', '4',
                    '5', '6', '7', '8', '9', '10', '11','image_features']]

# Print the resulting DataFrame
print(df.columns)


Index(['external_code', 'retail', 'category_encoded', 'fabric_encoded',
       'color_encoded', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10',
       '11', 'image_features'],
      dtype='object')


In [14]:
import pandas as pd

# Read the sales data frame from a csv file
sales_df = pd.read_csv("/content/visuelle2/sales.csv")

# Read the gtrends data frame from a csv file
gtrends_df = pd.read_csv("/content/visuelle2/vis2_gtrends_data.csv")
gtrends_df.head()


,date,long sleeve,culottes,miniskirt,short sleeves,printed shirt,short cardigan,solid color top,trapeze dress,sleeveless,...,scottish,milano stitch,devore,hron,ottoman,fluid,flamed,fluid polyviscous,shiny jersey,goose
0,2015-10-05,55.0,41.0,66.0,58.0,58.0,22.0,24.0,43.0,66.0,...,57,0,29,17,57,80,49,-1.0,0,34
1,2015-10-12,55.0,44.0,73.0,71.0,61.0,31.0,37.0,21.2,63.0,...,54,0,17,14,61,80,41,-1.0,0,39
2,2015-10-19,60.0,42.0,73.0,62.0,61.0,43.0,37.0,27.4,47.0,...,55,0,21,23,63,79,52,-1.0,33,41
3,2015-10-26,57.0,44.0,68.0,51.0,55.0,19.0,37.0,59.8,54.0,...,55,0,26,14,58,76,33,-1.0,32,37
4,2015-11-02,49.0,39.0,66.0,39.0,63.0,16.0,37.0,44.0,51.0,...,55,0,18,18,60,80,37,-1.0,98,38


In [23]:
print(sales_df.head())

   Unnamed: 0  external_code  retail season     category   color  \
0           0              5      36   SS17  long sleeve    grey   
1           1              2      51   SS17  long sleeve  violet   
2           2              5      10   SS17  long sleeve    grey   
3           3              9      41   SS17     culottes  yellow   
4           4              5      13   SS17  long sleeve    grey   

       image_path       fabric release_date  restock  ...    2    3    4    5  \
0  PE17/00005.png      acrylic   2016-11-28       22  ...  1.0  1.0  2.0  1.0   
1  PE17/00002.png      acrylic   2016-11-28       17  ...  1.0  0.0  0.0  2.0   
2  PE17/00005.png      acrylic   2016-11-28       15  ...  1.0  0.0  1.0  1.0   
3  PE17/00009.png  scuba crepe   2016-11-28       32  ...  1.0  1.0  0.0  0.0   
4  PE17/00005.png      acrylic   2016-11-28       26  ...  4.0  0.0  3.0  0.0   

     6    7    8    9   10   11  
0  0.0  0.0  2.0  0.0  0.0  0.0  
1  0.0  0.0  0.0  1.0  1.0  0.0  
2 

In [25]:
sales_df.iloc[1,:]

Unnamed: 0                    1
external_code                 2
retail                       51
season                     SS17
category            long sleeve
color                    violet
image_path       PE17/00002.png
fabric                  acrylic
release_date         2016-11-28
restock                      17
0                           1.0
1                           1.0
2                           1.0
3                           0.0
4                           0.0
5                           2.0
6                           0.0
7                           0.0
8                           0.0
9                           1.0
10                          1.0
11                          0.0
Name: 1, dtype: object

In [36]:
row = sales_df.iloc[106845,:]
cat, col, fab, start_date = row["category"], row["color"], row["fabric"], row["release_date"]
print(cat,'\n',col,'\n',fab)
start_date = pd.to_datetime(start_date)

# Calculate the end date as the start date plus 52 weeks
end_date = start_date + pd.DateOffset(weeks=52)

# Format the start and end dates as strings in YYYY-MM-DD format
start_date_str = start_date.strftime("%Y-%m-%d")
end_date_str = end_date.strftime("%Y-%m-%d")

print(start_date_str)
print(end_date_str)

gtrends_data = gtrends_df.loc[(gtrends_df["date"] >= start_date_str) & (gtrends_df["date"] <= end_date_str), [cat, col, fab]]

print(gtrends_data)

# Get the values of the three columns as numpy arrays
cat_gtrend = gtrends_data[cat].values
col_gtrend = gtrends_data[col].values
fab_gtrend = gtrends_data[fab].values
print(col_gtrend,'\n',cat_gtrend,'\n',fab_gtrend)
# Stack the three arrays into a 2D array
multitrends = np.vstack([cat_gtrend, col_gtrend, fab_gtrend])
print(multitrends)

medium coat 
 grey 
 foam rubber
2019-12-30
2020-12-28
Empty DataFrame
Columns: [medium coat, grey, foam rubber]
Index: []
[] 
 [] 
 []
[]


In [15]:
# Create an empty list to store the trends data
gtrends = []

# Iterate over each row of the sales data frame
for index, row in sales_df.iterrows():
    # Get the product information from the row
    cat, col, fab, start_date = row["category"], row["color"], row["fabric"], row["release_date"]

    # Convert the start date to a datetime object
    start_date = pd.to_datetime(start_date)

    # Calculate the end date as the start date plus 52 weeks
    end_date = start_date + pd.DateOffset(weeks=52)

    # Format the start and end dates as strings in YYYY-MM-DD format
    start_date_str = start_date.strftime("%Y-%m-%d")
    end_date_str = end_date.strftime("%Y-%m-%d")

    # Get the gtrends data for the corresponding category, color, and fabric
    # Use the loc function to filter the gtrends data frame by the date range and the columns
    gtrends_data = gtrends_df.loc[(gtrends_df["date"] >= start_date_str) & (gtrends_df["date"] <= end_date_str), [cat, col, fab]]

    # Get the values of the three columns as numpy arrays
    cat_gtrend = gtrends_data[cat].values
    col_gtrend = gtrends_data[col].values
    fab_gtrend = gtrends_data[fab].values

    # Stack the three arrays into a 2D array
    multitrends = np.vstack([cat_gtrend, col_gtrend, fab_gtrend])

    # Append the 2D array to the gtrends list
    gtrends.append(multitrends)

In [16]:
df["cat_gtrend"] = df.apply(lambda x: gtrends[x.name][0], axis=1)
df["col_gtrend"] = df.apply(lambda x: gtrends[x.name][1], axis=1)
df["fab_gtrend"] = df.apply(lambda x: gtrends[x.name][2], axis=1)

In [17]:
#save to csv
df.to_csv('dataFinal.csv')
df

,external_code,retail,category_encoded,fabric_encoded,color_encoded,0,1,2,3,4,...,6,7,8,9,10,11,image_features,cat_gtrend,col_gtrend,fab_gtrend
0,5,36,11,0,4,1.0,3.0,1.0,1.0,2.0,...,0.0,0.0,2.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[85.0, 84.0, 76.0, 65.0, 61.0, 59.0, 54.0, 53....","[20.0, 18.0, 18.0, 16.0, 18.0, 19.0, 17.0, 18....","[68.0, 69.0, 71.0, 78.0, 72.0, 69.0, 66.0, 69...."
1,2,51,11,0,7,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,"[0.0, 0.0, 0.0, 7.2603164, 0.0, 0.0]","[85.0, 84.0, 76.0, 65.0, 61.0, 59.0, 54.0, 53....","[56.0, 58.0, 59.0, 54.0, 58.0, 55.0, 58.0, 59....","[68.0, 69.0, 71.0, 78.0, 72.0, 69.0, 66.0, 69...."
2,5,10,11,0,4,1.0,3.0,1.0,0.0,1.0,...,1.0,1.0,1.0,0.0,0.0,1.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[85.0, 84.0, 76.0, 65.0, 61.0, 59.0, 54.0, 53....","[20.0, 18.0, 18.0, 16.0, 18.0, 19.0, 17.0, 18....","[68.0, 69.0, 71.0, 78.0, 72.0, 69.0, 66.0, 69...."
3,9,41,1,49,9,1.0,1.0,1.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[46.0, 52.0, 48.0, 47.0, 47.0, 48.0, 35.0, 56....","[72.0, 73.0, 71.0, 74.0, 74.0, 74.0, 75.0, 77....","[0.0, 17.0, 18.0, 0.0, 0.0, 0.0, 0.0, 17.0, 0...."
4,5,13,11,0,4,1.0,2.0,4.0,0.0,3.0,...,2.0,1.0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]","[85.0, 84.0, 76.0, 65.0, 61.0, 59.0, 54.0, 53....","[20.0, 18.0, 18.0, 16.0, 18.0, 19.0, 17.0, 18....","[68.0, 69.0, 71.0, 78.0, 72.0, 69.0, 66.0, 69...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106845,5504,51,14,18,4,2.0,1.0,1.0,3.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 11.0338125, 0.0]",[],[],[]
106846,5558,10,14,18,0,3.0,1.0,0.0,1.0,0.0,...,0.0,2.0,1.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 10.992317, 0.0]",[],[],[]
106847,4988,108,14,7,0,1.0,0.0,0.0,1.0,2.0,...,5.0,2.0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",[],[],[]
106848,4280,105,1,28,1,1.0,2.0,4.0,1.0,2.0,...,1.0,0.0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]",[],[],[]
